The goal of this notebook is to understand the orbit of the LMC in the vasiliev++ n-body models and how we may reasonably approximate this, even for short term evolution, for a galaxy like sculptor.

In [ ]:
using Printf
using CairoMakie
using Revise
using LilGuys, Arya
import NaNMath as nm

In [ ]:
using OrderedCollections

In [ ]:
using CSV, DataFrames

In [ ]:
using PythonCall
agama = pyimport("agama")
np = pyimport("numpy")

In [ ]:
figdir = "figures"

In [ ]:
include("agama_utils.jl")
include("agama_plots.jl")

In [ ]:
save = Makie.save

## ICs

Collect ICs

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)
gc_scl = LilGuys.transform(Galactocentric, icrs)

In [ ]:
icrs_lmc_me = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = 0.229,
    radial_velocity = 262,
    distance = 49.59
    )

icrs_lmc_me_err = ICRS(
    ra=0.3,
    dec=0.3,
    distance=0.54,
    pmra=0.2,
    pmdec=0.47,
    radial_velocity=3,
    )

In [ ]:
icrs_lmc = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
gc_lmc = LilGuys.transform(Galactocentric, icrs_lmc)

## Utilities

In [ ]:
# some simple python utilities
py2f(x) = pyconvert(Float64, x)
py2vec(x) = pyconvert(Vector{Float64}, x)
py2mat(x) = pyconvert(Matrix{Float64}, x)'

In [ ]:
"""
    make_lmc_pot(Mlmc, r_s; kwargs...)

Creates a LMC agama potential as truncated NFW similar to vasiliev24. 
Mlmc is the mass (in our code units) and r_s is the scale radius (also in our code units).
kwargs passed to agama.Potential.
"""
function make_lmc_pot(Mlmc, r_s; kwargs...)
    Φ_lmc = agama.Potential(;
            type="Spheroid", alpha=1, beta=3, gamma=1, 
            scaleradius = r_s,
            mass = Mlmc, 
            outercutoffradius = 10r_s, 
            cutoffStrength=4,
            kwargs...
        )
    return Φ_lmc
end


In [ ]:
"""
    calc_lmc_orbit(pot, gc_coord, kwargs...)

Computes an LMC orbit in the given potential with the present-day position `gc_coord`.
Returns an `Orbit` object.

# Arguments
- `Mlmc::function` a function which returns the mass of the LMC (our code units) given the time
- `reflex_motion::Bool=true` whether to include reflex motion in the calculation
- `dynamical_friction::Bool=true` whether to include (Chandrasakar dynamical friction) in the calculation
- `Λ = nothing`. The value of the argument of the Coloumb logarithm, 
controlling the scaling of Dynamical friction
- `σv`. A function returning the local halo velocity dispersion given distance from origin
- `vasiliev_units::Bool = false` If true, assumes the potential `pot` is in the alternate unit system in vasiliev++
- `time` Time to integrate to. If negative, integrates backwards in time. Code units.
- `r_s`
- `timestep`
Additional kwargs passed to `leap_frog`

"""
function calc_lmc_orbit(pot, gc_coord; 
        Mlmc = t->15,
        reflex_motion = true,
        dynamical_friction = true,
        Λ = nothing,
        σv = nothing,
        vasiliev_units = false,
        time = -10 / T2GYR, 
        r_s = 8.5 * (Mlmc / 10)^0.6,
        timestep = :adaptive,
        kwargs...
    )

    
    if vasiliev_units
        acc_scale = (V_V2KMS/ V2KMS)^2  * (V_R2KPC / R2KPC)^-1
        v_scale = V2KMS / V_V2KMS
        m_scale = M2MSUN / V_M2MSUN
    else
        v_scale = m_scale = acc_scale = 1
    end
    

    if σv === nothing
        calc_σv_interp(pot, log_r=LinRange(2, -2.0, 1000))
    end

    ρ(x) = py2f(pot.density(x))
    
    Φ_lmc = make_lmc_pot(Mlmc(0) * m_scale, r_s)


    f_fric(pos, vel, t) = dynamical_friction * a_dyn_friction(pos, vel, 
        r_s=r_s,
        σv=σv, ρ=ρ, M=Mlmc(t) * m_scale, Λ = sqrt(calc_r(pos) / 0.8r_s )
    )


    f_grav(pos, vel, t) = py2vec(
        pot.force(pos) - reflex_motion * Mlmc(t) / Mlmc(0) * Φ_lmc.force(-pos)
    )

    f_acc(pos, vel, t) = acc_scale * (f_grav(pos, vel*v_scale, t) .+ f_fric(pos, vel *v_scale, t) )

    orbit = leap_frog(gc_coord, f_acc; time=time, timestep=timestep, kwargs...)

    return orbit
end


In [ ]:
"""
    make_lmc_mw_pot_from_orbit

Given the MW potential and the orbit of the LMC,
returns an agama.Potential object representing the combined potential.
"""
function make_lmc_mw_pot_from_orbit(pot, orbit;
        reflex_motion=true,
        Mlmc=t->15,
        r_s=8.5 , 
        vasiliev_units=false,     
    )
    
    time = orbit.time

    if vasiliev_units
        m_scale = M2MSUN / V_M2MSUN
        time *= T2GYR / V_T2GYR
    else
        m_scale = 1
    end
    
    if orbit.time[2] < orbit.time[1]
        position = orbit.position[:, end:-1:1]
        time = reverse(time)
    end

    centre = vcat(time', position)

    centre = PyArray(centre')
    scale = PyArray(vcat(time',  Mlmc.(time)' ./ Mlmc(0), ones(length(time))')')

    pot_lmc = make_lmc_pot(m_scale * Mlmc(0), r_s, center=centre,scale=scale )

    
    if reflex_motion != 0
        N = length(time)
        a_reflex = pot_lmc.force(zeros(length(time), 3), t=time)
        nptime = PyMatrix(reshape(time,(:, 1)))
        mat_reflex = np.hstack([nptime, -a_reflex])        
        pot_reflex = agama.Potential(type="UniformAcceleration", file=mat_reflex)
        
        Φ = agama.Potential(pot, pot_lmc, pot_reflex)
    else
        Φ = agama.Potential(pot, pot_lmc)
    end

    return Φ
end

In [ ]:
"""
    make_lmc_mw_pot(pot, lmc_gc; kwargs...)

Given the MW potential and the LMC initial conditions, computes the orbit 
of the LMC and return the combined potential and the orbit.
Arguments are as in `calc_lmc_orbit`
"""
function make_lmc_mw_pot(pot, lmc_gc;
        reflex_motion=true,
        Mlmc=t->15,
        r_s=8.5 * (Mlmc / 10)^0.6, 
        vasiliev_units=false, 
        kwargs...
    
    )
    orbit = calc_lmc_orbit(pot, lmc_gc; Mlmc=Mlmc, r_s=r_s, vasiliev_units=vasiliev_units, reflex_motion=reflex_motion, kwargs...)
    Φ = make_lmc_mw_pot_from_orbit(pot, orbit; Mlmc=Mlmc, r_s=r_s, vasiliev_units=vasiliev_units, reflex_motion=reflex_motion)

    return Φ, orbit
end

In [ ]:
function calc_σv_func(pot_halo)
    gridr = np.logspace(1, 3, 16)

    df = agama.DistributionFunction(type="quasispherical", potential=pot_halo)
    gm = agama.GalaxyModel(pot_halo, df)

    x = np.column_stack((gridr, gridr*0, gridr*0))
    
    sigmas = py2mat(gm.moments(x, dens=false, vel=false))[1, :] .^ 0.5

    println("sigmas = ", sigmas)
    sigmafnc = agama.Spline(gridr, sigmas)
    return sigmafnc
end

# Alla Vasiliev+21

I have the potential from the Vasiliev et al. (V+21, 2021) paper "Tango for Three...", stored in agama/potentials/vasiliev+2021.
The components of this potential are:

- Multipole expansion of evolving N-body MW potential
- Moving & Evolving multipole expansion of an LMC potential from their N-body simulation
- The non-inertial acceleration of the MW

My goal with the plots below is to investigate these effects and how each one in turn influences both the orbtits of the LMC and Scl. To 2st order, each effect is important but likely a full N-body simulation is required to capture the full nuance of interacting galaxies.

In [ ]:
vasiliev21_frozen = load_agama_potential("vasiliev+21/potential_nolmc.ini")
vasiliev21 = load_agama_potential("vasiliev+21/potential_evolving.ini")
vasiliev21_lmc = load_agama_potential("vasiliev+21/potential_lmc_only.ini")

In [ ]:
function read_lmc_traj(filename)
    lmc_traj = CSV.read(filename, DataFrame, delim=" ", header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])
    
    lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))
    
    
    orbit_lmc_act = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
        )
end


In [ ]:
function read_lmc_acc(filename)
    acc_v21 = CSV.read(filename, DataFrame, delim=" ", header = [:time, :x, :y, :z])
end

In [ ]:
# loads in trajectory of lmc in Vasiliev 2021
lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/trajlmc.txt"
orbit_lmc_act = read_lmc_traj(lmc_file)

In [ ]:
filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev+21/accel.txt"
acc_lmc_act = read_lmc_acc(filename)

In [ ]:
lmc_gc = LilGuys.transform(LilGuys.Galactocentric, lmc_coord)

In [ ]:
lmc_traj[lmc_traj.time .== 0, :]

In [ ]:
LilGuys.position_of(lmc_gc)

In [ ]:
LilGuys.velocity_of(lmc_gc)

## Orbits

In [ ]:
Φ_v21_simple = make_lmc_mw_pot_from_orbit(vasiliev21_frozen, orbit_lmc_act,
    Mlmc=15, 
    vasiliev_units=true,
    reflex_motion=true,
)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(vasiliev21_frozen, lmc_gc,
    Mlmc=15, 
    σv=x->120, 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true
)

### Reflex acceleration

In [ ]:
times = acc_v21.time

orbit_lmc_act_resamp = resample(orbit_lmc_act, times * V_T2GYR / T2GYR)

In [ ]:
Φ_lmc_stat = make_lmc_pot(15 * M2MSUN / V_M2MSUN, 10.84)

In [ ]:
Φ_lmc_stat.totalMass() * V_M2MSUN / M2MSUN

In [ ]:
vasiliev21_frozen.totalMass() * V_M2MSUN / M2MSUN

In [ ]:
Φ_lmc_evolving = Φ_evolving[1]

In [ ]:

acc_me = -Φ_lmc_evolving.force(zeros(length(times), 3),  t=times) |> py2mat
acc_me_fmw = vasiliev21_frozen.force(orbit_lmc_act_resamp.position', t=times) *  15 / 80.19 |> py2mat

acc_me_act_orbit = Φ_lmc_stat.force(orbit_lmc_act_resamp.position',  t=times) |>py2mat

# the actual acceleration of the model
acc_v21_mat = hcat(acc_v21.x, acc_v21.y, acc_v21.z)'


In [ ]:
fig = LilGuys.Plots.plot_xyz(acc_me, acc_me_fmw, acc_me_act_orbit, acc_v21_mat, 
    labels = ["my orbit", "rescaled mw-lmc force", "V+21 simple", "V+21 actual"],
    units="",
    xlabel="a_x" ,
    ylabel="a_y",
    zlabel="a_z",
)
fig

In [ ]:
vasiliev21

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel="time / Gyr", ylabel = L"|a_\textrm{MW}|")

lines!(times, calc_r(acc_me), label="my orbit")
lines!(times, calc_r(acc_me_fmw), label="rescaled mw-lmc force")
lines!(times, calc_r(acc_me_act_orbit), label="V+21 simple")
lines!(times, calc_r(acc_v21_mat), label="V+21 actual")

axislegend(position=:lt)
fig

In the plot above, I compare several different methods for calculating the reflex acceleration of the milkyway.
- `my orbit` is the reflex motion calculated by the force at zero due to the LMC on my fiducial LMC orbit
- `rescaled mw-lmc force` uses the V+21 orbit but instead calculates the force of the MW on the LMC and rescales by the mass ratio
- `V+21 simple` uses the same method as `my orbit` except along the LMC orbit of V+21
- `V+21` contains the actual N-body model's acceleration of the MW

COmparing these models, all are similer except the rescaled mw-lmc force underestimates the acceleration. The actual acceleration deviates from both my orbit and the `V+21 simple` models especially past t=0, but for past times, the magnitude of acceleration is fairly close for each model.

### Orbits

In [ ]:
orbit_scl = calc_orbit(gc_scl, Φ_evolving, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_v21_simple = calc_orbit(gc_scl, Φ_v21_simple, time=-5, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl_nolmc = calc_orbit(gc_scl, vasiliev21_frozen, time=-5, units=:vasiliev)

In [ ]:
v21_noref = agama.Potential(vasiliev21[0], vasiliev21[1], vasiliev21[3], vasiliev21[4])

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, vasiliev21, time=-5, units=:vasiliev, N=10001)

In [ ]:
orbit_lmc_act = resample(orbit_lmc_act, orbit_scl.time)

In [ ]:
orbit_scl_lmc  = orbit_scl - orbit_lmc 
orbit_scl_lmc_act = orbit_scl_act - orbit_lmc_act
orbit_scl_lmc_simple = orbit_scl_v21_simple - orbit_lmc_act

In [ ]:
plot_y_z([
    "vasiliev+21" => orbit_scl_lmc_act,
    "vasiliev+21 simple" => orbit_scl_lmc_simple,
        "approx" => orbit_scl_lmc,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_act,
    "scl me" => orbit_scl,
    "scl vasiliev+21" => orbit_scl_act,
    "scl vasiliev+21 simple" => orbit_scl_v21_simple,

        ])

In [ ]:
orbits = [
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_act,
    "scl me" => orbit_scl,
    "scl vasiliev+21" => orbit_scl_act,
    "scl vasiliev+21 simple" => orbit_scl_v21_simple,

        ]

In [ ]:
plot_xyz(orbits)

In [ ]:
lmc_me = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=true,
    timestep=:adaptive
)

lmc_point = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=false
)

lmc_dyf = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=true, reflex_motion=false
)


lmc_ref = calc_lmc_orbit(vasiliev21_frozen, lmc_gc, 
    vasiliev_units=true, σv=x->120,
    dynamical_friction=false, reflex_motion=true
)

In [ ]:
orbits = [
    "point" => lmc_point,
    "+reflex" => lmc_ref,
    "+dyn fric" => lmc_dyf,
    "reflex & dyn friction" => lmc_me,
    "vasiliev+2021" => orbit_lmc_act,
    ];

In [ ]:
plot_xyz(orbits)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])
plot_v_circ!(Φ_lmc_stat, vasiliev_units=true, log=false, log_r=log10.(LinRange(0.1, 50, 100)))

fig

In [ ]:
Φ_lmc_stat.density([30, 0, 0])

In [ ]:
fig = plot_r_t(orbits, legend=false)

lines!(lmc_orbit.time * T2GYR, r_scl_lmc, label="Scl-LMC ")
axislegend()
fig

In [ ]:
log10(LilGuys.kpc_to_arcmin(LilGuys.calc_break_radius(8.0 / V2KMS, 0.1 / T2GYR), 83.2))

In [ ]:
log10(20)

In [ ]:
fig, ax = FigAxis(
    xlabel = "time / Gyr",
    ylabel = "distance from Scl to LMC / kpc",
    limits=(nothing, nothing, 0, nothing),
    )

lines!(lmc_orbit.time * T2GYR, r_scl_lmc)

fig

In [ ]:
lmc_orbit.time[argmin(r_scl_lmc)] * T2GYR

In [ ]:
minimum(r_scl_lmc)

In [ ]:
lmc_orbit.time ./ scl_lmc_orbit.time

In [ ]:
plot_v_circ(ep20)

In [ ]:
plot_v_circ(vasiliev21, vasiliev_units=true)

# Alla Vasiliev 2024

Similar to above but for ...

- Multipole expansion of evolving N-body MW potential
- Moving & Evolving multipole expansion of an LMC potential from their N-body simulation
- The non-inertial acceleration of the MW

My goal with the plots below is to investigate these effects and how each one in turn influences both the orbtits of the LMC and Scl. To 2st order, each effect is important but likely a full N-body simulation is required to capture the full nuance of interacting galaxies.

In [ ]:
using CSV, DataFrames

In [ ]:
"""
    load_lmc_traj(modelname)

Reads in the LMC trajectory as an `Orbit` object
given the name of the `vasiliev24` model (e.g. "L3M11")
"""
function load_lmc_traj(modelname::String)
    lmc_file = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/trajlmc.txt"
    lmc_traj = CSV.read(lmc_file, DataFrame, delim=" ", ignorerepeated=true, header = [:time, :x, :y, :z, :v_x, :v_y, :v_z])
    lmc_traj = lmc_traj
    lmc_traj = DataFrame(reverse(eachrow(lmc_traj)))
        
    orbit_lmc_act = Orbit(time=lmc_traj.time  * V_T2GYR/ T2GYR, 
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = [lmc_traj.v_x lmc_traj.v_y lmc_traj.v_z]' * V_V2KMS / V2KMS
        )
    return orbit_lmc_act
end

In [ ]:
"""
    load_accel(modelname)

Reads in the MW reflex acceleration 
given the name of the `vasiliev24` model (e.g. "L3M11")
"""
function load_accel(modelname)
    filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/accel.txt"
    df =  CSV.read(filename, DataFrame, delim=" ", ignorerepeated=true, comment="#", header=["time", "x", "y", "z"])
    acc_scale = (V_V2KMS/ V2KMS)^2  * (V_R2KPC / R2KPC)^-1
    df.x .*= acc_scale
    df.y .*= acc_scale
    df.z .*= acc_scale
    df.time * V_T2GYR, [df.x df.y df.z]'
end

In [ ]:
"""
    load_bound_mass(modelname)

Reads in the LMC bound mass
given the name of the `vasiliev24` model (e.g. "L3M11")
"""
function load_bound_mass(modelname)
    filename = ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/boundmass.txt"
    df =  CSV.read(filename, DataFrame, delim=" ", ignorerepeated=true, comment="#", header=["time", "mass"])
    df.mass ./= M2MSUN
    df.time .*= V_T2GYR
    df
end

In [ ]:
function get_acc(Φ, time)
    return Φ[-1].force(np.zeros((length(time), 3)), t=time / V_T2GYR) .* (V_V2KMS / V2KMS)^2  |> py2mat
end

In [ ]:
function get_∇acc(Φ, time, position=nothing)
    if position === nothing
        position = np.zeros((length(time), 3))
    else
        position = np.array(position')
    end

    ∇acc =  Φ.eval(position, t=time, der=true)
    return py2mat(∇acc)
end

In [ ]:
lmc_coord = ICRS(ra = 81,
    dec = -69.75, 
    pmra = 1.8,
    pmdec = 0.35,
    radial_velocity = 260,
    distance = 50.
    )

In [ ]:
# closer to what is actually used in the paper
lmc_gc = LilGuys.Galactocentric( -0.61,  -41.02,  -26.83,  -69.84,  -221.66,  214.12)

### other units

In [ ]:
pot_frozen = agama.Potential( 
    pydict(type="Spheroid", mass=1.2, scaleRadius=0.2, outerCutoffRadius=1.8, gamma=0, beta=1.8),
    pydict(type="Disk", surfaceDensity=0.088431375, scaleRadius=3.0, scaleHeight=-0.25),
    pydict(type="Spheroid", densityNorm=0.97e-3, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1),
    )

In [ ]:
lmc_traj = CSV.read("/arc7/home/dboyea/dwarfs/agama/potentials/vasiliev24/scripts/lmc_orbit.csv", DataFrame)

orbit_lmc_agama = Orbit(time=lmc_traj.t * V_T2GYR /T2GYR ,
        position = [lmc_traj.x lmc_traj.y lmc_traj.z]',
        velocity = ([lmc_traj.vx lmc_traj.vy lmc_traj.vz] / V2KMS)' 
        )

In [ ]:
pwd()

In [ ]:
gridr = numpy.logspace(1, 3, 32)

pot_halo = agama.Potential(type="Spheroid", densityNorm=0.97e-3, scaleRadius=16.5, outerCutoffRadius=500, cutoffStrength=4, beta=3, gamma=1)

df = agama.GalaxyModel(pot_halo, agama.DistributionFunction(type="quasispherical", potential=pot_halo))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
gridr

In [ ]:
df.totalMass()

In [ ]:
df.totalMass()

In [ ]:
sigmas * V2KMS

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=false, 
    reflex_motion=true, 
    dynamical_friction=true
)

In [ ]:
extrema(calc_r(orbit_lmc.position))

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "agama" => orbit_lmc_agama,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "lmc vasiliev+21" => orbit_lmc_agama,

        ])

almost exactly matches agama code. Slight differences are likely due to how agama internally grids data and that we use a different unit system than in the notebook. 

### Bound mass

In [ ]:
boundmass = OrderedDict(
    "L2M10" => load_bound_mass("L2M10"),
    "L2M11" => load_bound_mass("L2M11"),
    "L3M10" => load_bound_mass("L3M10"),
    "L3M11" => load_bound_mass("L3M11"),
    );

In [ ]:
fig = Figure()
ax = Axis(fig[1,1],
    xlabel = "time / Gyr",
    ylabel = L"LMC bound mass / $10^{10}\,$M$_\odot$"
)

for (label, df) in boundmass
    plot!(df.time, df.mass, label = label)
end

axislegend()

fig

### L3M11

In [ ]:
modelname = "L3M11"

In [ ]:
pot_halo = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_halo.ini")
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")
pot_evolving = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential.ini")


In [ ]:
df = load_bound_mass(modelname)
boundmass_f = LilGuys.lerp(df.time / T2GYR, df.mass)

orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
sigmafnc = calc_σv_func(pot_halo)

In [ ]:
default_params = Dict(
    :Mlmc => x->30, 
    :r_s => 11.7,
    :σv => x->py2f(sigmafnc(x)), 
    :vasiliev_units => true, 
    :reflex_motion => true, 
    :dynamical_friction => true,
    :dt_max => 5
)

In [ ]:
potential_parameters = OrderedDict(
    "m15" => Dict(
        :Mlmc => x->15, 
    ),
    "m20" => Dict(
        :Mlmc => x->20, 
    ),
    "m20s" =>  Dict( 
        :Mlmc => x->20, 
        :r_s => 11.7,
        :reflex_motion=>false,
        :dynamical_friction => false,
    ),
    "m30" => Dict(
        :Mlmc => x->30, 
    ),
    # "m_evol" => Dict(
    #     :Mlmc => boundmass_f, 
    # ),
    # "tuned" => Dict(
    #     :Mlmc => x->15, 
    #     :dynamical_friction => 1.15,
    # ),    
)


In [ ]:
# Φ_evolving_tuned, orbit_lmc_tuned  = make_lmc_mw_pot(pot_frozen, lmc_gc,
#     Mlmc=x->15, 
#     r_s=11.7,
#     σv= x->py2f(sigmafnc(x)), 
#     vasiliev_units=true, 
#     reflex_motion=true, 
#     dynamical_friction=1.15,
#     dt_max=5,
# )

In [ ]:
Φs = OrderedDict()
orbits_lmc = OrderedDict()

for (label, params) in potential_parameters
    println("calculating $label")
    kwargs = merge(default_params, params)
    Φ, orbit = make_lmc_mw_pot(pot_frozen, lmc_gc; kwargs...)
    Φs[label] = Φ
    orbits_lmc[label] = orbit

end

In [ ]:
mw_acc = OrderedDict()

for label in keys(potential_parameters)
    acc = get_acc(Φs[label], orbits_lmc[label].time .* T2GYR / V_T2GYR)
    if (:reflex_motion ∈ keys(potential_parameters[label]) ) && !potential_parameters[label][:reflex_motion]
       acc .*= 0
    end
    mw_acc[label] = acc
end

In [ ]:
lmc_traj = CSV.read("/arc7/home/dboyea/dwarfs/agama/potentials/vasiliev24/scripts/L3M11_orbit.csv", DataFrame)
t, acc = load_accel(modelname)

orbits_lmc["vasiliev24"] = orbit_lmc_act
mw_acc["vasiliev24"] = acc
Φs["vasiliev24"] = pot_evolving

In [ ]:
plot_y_z(orbits_lmc)

In [ ]:
plot_r_t(orbits_lmc)

In [ ]:
LilGuys.plot_xyz(collect(values(mw_acc))..., labels = collect(keys(mw_acc)), units=" / kpc T0^-2")

In [ ]:
orbits_scl = OrderedDict()

for (key, Φ) in Φs
    orbit = calc_orbit(gc_scl, Φ, time=-5, units=:vasiliev, N=10_001)
    orbits_scl[key] = orbit
end

In [ ]:
plot_r_t(orbits_scl)

In [ ]:
plot_y_z(orbits_scl)

In [ ]:
orbits_lmc

In [ ]:
fig = Figure()


ax1 = Axis(fig[1,1], 
    ylabel = "x Scl - x LMC",
)


for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    y = orbit.position[1, :] .- o_lmc.position[1, :]
    lines!(x * T2GYR, y, label=label)
end

ax2 = Axis(fig[2,1],
    ylabel = "y Scl - y LMC"
)

for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    y = orbit.position[2, :] .- o_lmc.position[2, :]
    lines!(x * T2GYR, y)
end


ax3 = Axis(fig[3,1],
    ylabel = "z Scl - z LMC",
    xlabel = "time / Gyr"
)

for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    y = orbit.position[3, :] .- o_lmc.position[3, :]
    lines!(x * T2GYR, y)
end


linkaxes!(ax1, ax2, ax3)
xlims!(ax3, -2, 0)
for ax in [ax1, ax2]
    hidexdecorations!(ax, grid=false, ticks=false)
end

Legend(fig[1,2], ax1)
fig

In [ ]:
fig = Figure()


ax1 = Axis(fig[1,1],
    limits=(-5, 0, 0, 220),
    xlabel = "time / Gyr",
    ylabel = "Scl—LMC distance"
)


for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    r = calc_r(orbit.position, o_lmc.position)
    lines!(x * T2GYR, r, label=label)
end


axislegend(position=:lb)
fig

In [ ]:
fig = Figure()


ax1 = Axis(fig[1,1],
    xlabel = "time / Gyr",
    ylabel = L"Scl - LMC relative velocity / km\,s$^{-1}$"
)


for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    r = calc_r(orbit.velocity, o_lmc.velocity)
    lines!(x * T2GYR, r * V2KMS, label=label)
end

axislegend(position=:lt)


fig

In [ ]:
fig = Figure()


ax1 = Axis(fig[1,1],
    xlabel = "time / Gyr",
    ylabel = L"Scl - MW relative velocity / km\,s$^{-1}$"
)


for (label, orbit) in orbits_scl
    x = orbit.time
    o_lmc = resample(orbits_lmc[label], x)
    r = calc_r(orbit.velocity)
    lines!(x * T2GYR, r * V2KMS, label=label)
end

axislegend(position=:lt)


fig

In [ ]:
fig = Figure()


ax1 = Axis(fig[1,1],
    xlabel = "time / Gyr",
    ylabel = L"tidal stress ? ($-\textrm{trace}(\textbf{H} Φ)$)",
    limits=(-2, 0, nothing, nothing),
)


for (label, orbit) in orbits_scl
    pot = Φs[label]
    pos = orbit.position[:, 1:10:end]
    t = orbit.time[1:10:end] * T2GYR / V_T2GYR
    
    df = get_∇acc(pot, t, pos)

    ax, ay, az, axy, ayz, azx = eachrow(df)
    da_tot = @. ax^2 + ay^2 + az^2 #+ 2ax^2 + 2ay^2 + 2az^2
    da_tot = @. -(ax + ay + az)
    lines!(t * V_T2GYR, da_tot, label=label)
    
end

axislegend(position=:lt)


fig

In [ ]:
fig = Figure(size=(800, 600))


ax1 = Axis(fig[1,1],
    ylabel=L"\partial_x^2 \Phi",
)
ax2 = Axis(fig[2,1],
    ylabel=L"\partial_y^2 \Phi"
)
ax3 = Axis(fig[3,1],
    ylabel=L"\partial_z^2 \Phi",
    xlabel = "time / Gyr"
)
ax4 = Axis(fig[1,2],
    ylabel=L"\partial_x\partial_y \Phi"
)
ax5 = Axis(fig[2,2],
    ylabel=L"\partial_y\partial_z \Phi"
)
ax6 = Axis(fig[3,2],
    ylabel=L"\partial_z\partial_x \Phi",
    xlabel = "time / Gyr"
)


for (label, orbit) in orbits_scl
    
    pot = Φs[label]
    pos = orbit.position[:, 1:10:end]
    t = orbit.time[1:10:end] * T2GYR / V_T2GYR
    filt = t .> -1

    println(sum(filt))
    t = t[filt]
    pos = pos[:, filt]
    println(pot)
    df = get_∇acc(pot, t, pos)

    ax, ay, az, axy, ayz, azx = eachrow(df)

    if label == "m20s" 
        plot_kwargs = (; linestyle=:dot)
    else
        plot_kwargs = (;)
    end
    
    lines!(ax1, t, ax, label=label; plot_kwargs...)
    lines!(ax2, t, ay; plot_kwargs...)
    lines!(ax3, t, az; plot_kwargs...)
    
    lines!(ax4, t, axy; plot_kwargs...)
    lines!(ax5, t, ayz; plot_kwargs...)
    lines!(ax6, t, azx; plot_kwargs...)
end

linkxaxes!(ax1, ax2, ax3, ax4, ax5, ax6)
hidexdecorations!(ax1, grid=false, ticks=false)
hidexdecorations!(ax2, grid=false, ticks=false)
hidexdecorations!(ax4, grid=false, ticks=false)
hidexdecorations!(ax5, grid=false, ticks=false)

Legend(fig[2, 3], ax1)


fig

### L2M10

In [ ]:
modelname = "L2M10"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
df = load_bound_mass(modelname)
boundmass_L2M10 = LilGuys.lerp(df.time, df.mass)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
    dt_max=10
)

In [ ]:
Φ_evolving_massevolv, orbit_lmc_massevolv = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=boundmass_L2M10, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.3,
    dt_max=5,

)

In [ ]:
Φ_evolving_tuned, orbit_lmc_tuned  = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=x->13, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=1.2,
    dt_max=5,
)

In [ ]:
orbits = [
    "lmc me" => orbit_lmc,
    "lmc tuned" => orbit_lmc_tuned,
    "lmc + mass evol" => orbit_lmc_massevolv,
    "vasiliev24" => orbit_lmc_act,
        ]

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

In [ ]:
time, acc = load_accel(modelname)

In [ ]:
boundmass_L2M10(0)

In [ ]:
acc_tuned = get_acc(Φ_evolving_tuned, time)
acc_massevolv = get_acc(Φ_evolving_massevolv, time)
acc_first = get_acc(Φ_evolving, time)


In [ ]:
LP.plot_xyz(acc_first, acc_tuned, acc_massevolv, acc, labels = ["me", "tuned", "mass evolv", "act"])

In [ ]:
accs = [
    "me" => acc_first, 
    "tuned" => acc_tuned,
    "mass evolv" => acc_massevolv, 
    "act" => acc
    ]

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], xlabel = "time", ylabel = "MW acceleration")

for (label, a) in accs
    lines!(time * V_T2GYR, calc_r(a), label=label)

end
axislegend(position=:lt)
fig

In [ ]:
orbit_scl_tuned = calc_orbit(gc_scl, Φ_evolving_tuned, time=-10, units=:vasiliev, N=10_001)

In [ ]:
orbit_scl = calc_orbit(gc_scl, Φ_evolving, time=-10, units=:vasiliev)

In [ ]:
orbit_scl_massevolv = calc_orbit(gc_scl, Φ_evolving_massevolv, time=-10, units=:vasiliev)

In [ ]:
orbit_scl_act = calc_orbit(gc_scl, pot_evolving, time=-10, units=:vasiliev, N=10001)

In [ ]:
orbits = [
    "lmc " => orbit_scl,
    "lmc tuned" => orbit_scl_tuned,
    "lmc + mass evol" => orbit_scl_massevolv,
    "vasiliev24" => orbit_scl_act,
        ]

In [ ]:
plot_r_t(orbits)

In [ ]:
plot_y_z(orbits)

### L2M11

In [ ]:
modelname = "L2M11"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=20, 
    r_s=9,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
)

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

### L3M10

In [ ]:
modelname = "L3M10"

In [ ]:
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")

In [ ]:
orbit_lmc_act = load_lmc_traj(modelname)

In [ ]:
gridr = numpy.logspace(1, 3, 16)

df = agama.GalaxyModel(pot_frozen, agama.DistributionFunction(type="quasispherical", potential=pot_frozen))
sigmas = py2mat(df.moments(numpy.column_stack((gridr, gridr*0, gridr*0)), dens=false, vel=false))[1, :] .^ 0.5

sigmafnc = agama.Spline(gridr, sigmas)

In [ ]:
Φ_evolving, orbit_lmc = make_lmc_mw_pot(pot_frozen, lmc_gc,
    Mlmc=30, 
    r_s=11.7,
    σv= x->py2f(sigmafnc(x)), 
    vasiliev_units=true, 
    reflex_motion=true, 
    dynamical_friction=true,
)

In [ ]:
plot_r_t([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

In [ ]:
plot_y_z([
    "lmc me" => orbit_lmc,
    "vasiliev24" => orbit_lmc_act,

        ])

## Visualizing models

In [ ]:
log_r = LinRange(-1, 3, 100)
a = [1,0, 0]


fig = Figure()
ax = Axis(fig[1,1],
    xlabel = L"$r$ / kpc",
    ylabel = L"$\langle \rho_{DM} \rangle$ / $10^{10}$\,M$_\odot$\,kpc$^{-3}$",
    limits = (-1, 3, -12, 0),
)

sampled_points = LilGuys.rand_unit(100)

for (i, t) in enumerate(times)
    x0 = [0, 0, 0]
    ρs = []
    for a in eachcol(sampled_points)
        xs = x0' .+ a' .* 10 .^ log_r
        
        ρ = vasiliev24_lmc.density(xs, t=t ./ V_T2GYR) |> py2vec
        push!(ρs, ρ)
    end

    ρ = sum(ρs) ./ length(ρs)

    ρ_err = [sqrt(sum(([x[j] for x in ρs] .- ρ[j]) .^ 2) ./ length(ρs)) for j in eachindex(ρs)]


    if i == length(t)
        println("std = ", ρ_err ./ ρ)
        
    end

    
    ρ .*= V_M2MSUN / M2MSUN

    lines!(log_r, nm.log10.(ρ), color=t, colorrange=extrema(times))
end

lines!(log_r, log10.(calc_ρ.(nfw_model, 10 .^ log_r)), linestyle=:dot, color=COLORS[3])
text!(2.5, -7, text="NFW", rotation=-π/4, color=COLORS[3])

Colorbar(fig[1,2], colorrange=extrema(times), label="time / Gyr")

save("$figdir/L3M11_rho_lmc.pdf", fig)
fig

##  EP2021

In [ ]:
V_T2GYR

In [ ]:
ep20 = load_agama_potential("EP2020.ini")
galpy14 = load_agama_potential("galpy_2014.ini") 

In [ ]:
pot = ep20

In [ ]:
σv = calc_σv_interp(pot)

In [ ]:
lines(log10.(σv.x), σv.y * V2KMS)

In [ ]:
orbit_nofric = calc_orbit(gc, pot, time=-10/T2GYR)

In [ ]:
Mmw = pyconvert(Float64, pot.enclosedMass(300))

In [ ]:
Mlmc = 10.

In [ ]:
f_fric(pos, vel) = a_dyn_friction(pos, vel, r_s=10, σv=σv, ρ=x->pyconvert(Float64, pot.density(x)), M=Mlmc)

In [ ]:
use_noninacc = 1

In [ ]:
f_grav(pos, vel) = (1 + use_noninacc * Mlmc/Mmw) * pyconvert(Vector{Float64}, pot.force(pos)) 

In [ ]:
f_acc(pos, vel) =  f_grav(pos, vel) .+ f_fric(pos, vel)

In [ ]:
orbit_2 = leap_frog(gc, f_acc, dt_max=1)

In [ ]:
fig, ax, _ = plot_r_t(orbit_nofric)
plot_r_t!(ax, orbit_2)

fig

In [ ]:
fig, ax, p = plot_y_z(orbit_nofric)
lines!(orbit_2.position[2, :], orbit_2.position[3, :])

fig


# MC with LMC

In [ ]:
obs_props_filename = ENV["DWARFS_ROOT"] * "/observations/sculptor/observed_properties.toml"
icrs = LilGuys.coord_from_file(obs_props_filename)
icrs_err = LilGuys.coord_err_from_file(obs_props_filename)
gc_scl = LilGuys.transform(Galactocentric, icrs)

In [ ]:
modelname = "L3M11"

In [ ]:
pot_halo = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_halo.ini")
pot_frozen = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential_mw_init.ini")
pot_evolving = agama.Potential(ENV["DWARFS_ROOT"] * "/agama/potentials/vasiliev24/$modelname/potential.ini")


In [ ]:
icrs_err

In [ ]:
function scl_lmc_sample_and_orbit(Φ, scl, scl_err, lmc, lmc_err; Mlmc=20, N=1000, Mlmc_err=0.2, a_fric=1.1, a_fric_err=0.2, kwargs...)
    sigmafnc = calc_σv_func(pot_halo)
    
    icrs_samples = LilGuys.rand_coords(lmc, lmc_err, N)
    gc_samples = LilGuys.transform.(Galactocentric, icrs_samples)
    icrs_scl_samples = LilGuys.rand_coords(scl, scl_err, N);
    gc_scl_samples = LilGuys.transform.(Galactocentric, icrs_scl_samples);
    Mlmcs = Mlmc * 10 .^ (Mlmc_err * randn(N))

    icrs_df = LilGuys.to_frame(icrs_samples)
    icrs_scl_df = LilGuys.to_frame(icrs_scl_samples)
    a_fric_samples = a_fric .* 10 .^ (a_fric_err .* randn(N))


    orbits = Orbit[]
    orbits_scl = Orbit[]
    Φs = []
    
    for i in 1:N
        print("orbit $i")
        gc = gc_samples[i]
        m = Mlmcs[i]
        
        Φ_new, orbit = make_lmc_mw_pot(pot_frozen, gc, 
            time=-1 / T2GYR, vasiliev_units=true, Mlmc=t->m,  σv=x->py2f(sigmafnc(x)), 
            r_s=11.7, reflex_motion=true, dynamical_friction=a_fric_samples[i], dt_max=5) 
        
        push!(orbits, orbit)
        push!(Φs, Φ_new)

        
        gc = gc_scl_samples[i]
        orbit = calc_orbit(gc, Φs[i], time=orbits[i].time * T2GYR , units=:vasiliev);
        push!(orbits_scl, orbit)
    end

    orbits_scl_lmc = [orbits_scl[i] - orbits[i] for i in eachindex(orbits)]


    # properties
    peri_lmc = Vector{Float64}(undef, N)
    peris = Vector{Float64}(undef, N)
    t_peri = Vector{Float64}(undef, N)
    t_peri_lmc = Vector{Float64}(undef, N)
    
    for i in 1:N
        r = calc_r(orbits_scl[i].position)
        idx = argmin(r)
        t_peri[i] = orbits_scl[i].time[idx]
        peris[i] = r[idx]
    
        r = calc_r(orbits_scl_lmc[i].position)
        idx = argmin(r)
        t_peri_lmc[i] = orbits_scl_lmc[i].time[idx]
        peri_lmc[i] = r[idx]
    end

    # df all
    df_all = copy(icrs_scl_df)
    for symbol in names(df_all)
        df_all[:, "$(symbol)_lmc"] = icrs_df[:, symbol]
    end
    
    df_all[:, "t_peri"] = t_peri * T2GYR
    df_all[:, "t_peri_lmc"] = t_peri_lmc * T2GYR
    df_all[:, "peri"] = peris
    df_all[:, "peri_lmc"] = peri_lmc
    df_all[:, "Mlmc"] = Mlmcs
    df_all[:, "a_fric"] = a_fric_samples
        
    return orbits, orbits_scl, orbits_scl_lmc, Φs, df_all
end

In [ ]:
orbits, orbits_scl, orbits_scl_lmc, Φs, df_all = scl_lmc_sample_and_orbit(pot_frozen, icrs, icrs_err, icrs_lmc_me, icrs_lmc_me_err, N=1000)

In [ ]:
function plot_orbits(orbits, Nmax=length(orbits))
    fig = plot_r_t([i=>o for (i, o) in enumerate(orbits[1:Nmax])], alpha=0.2, color=:black, legend=false)

    display(fig)


    fig = Figure()
    ax = Axis(fig[1,1], 
    xlabel = "y", 
    ylabel = "z",
    )

    for orbit in orbits[1:Nmax]
        lines!(orbit.position[2, :], orbit.position[3, :], color=:black, alpha=0.2)
    end

    fig

end
    

In [ ]:
plot_orbits(orbits, 100)

In [ ]:
plot_orbits(orbits_scl, 100)

In [ ]:
plot_orbits(orbits_scl_lmc, 100)

In [ ]:
plot_labels = OrderedDict(
    :ra => "RA  / degrees",
    :dec => "Dec / degrees",
    :pmra => L"$\mu_{\alpha *}$ / mas\,yr$^{-1}$",
    :pmdec => L"$\mu_{\delta}$ / mas\,yr$^{-1}$",
    :radial_velocity => L"$v_\textrm{los}$ / km\,s$^{-1}$",
    :distance => "distance / kpc",
    :ra_lmc => "RA LMC / degrees",
    :dec_lmc => "Dec LMC / degrees",
    :pmra_lmc => L"$\mu_{\alpha *, lmc}$ / mas\,yr$^{-1}$",
    :pmdec_lmc => L"$\mu_{\delta, lmc}$ / mas\,yr$^{-1}$",
    :radial_velocity_lmc => L"$v_\textrm{los, lmc}$ / km\,s$^{-1}$",
    :distance_lmc => "distance lmc / kpc",
    :peri => "MW pericentre / kpc",
    :t_peri => "time of peri / Gyr",
    :peri_lmc => "Scl—LMC pericentre / kpc",
    :t_peri_lmc => "time of Scl-LMC peri / Gyr",
    :Mlmc => "lmc Mass",
    :a_fric => "dyn fric"
    )

In [ ]:
function plot_hists(df_all)
    
    fig = Figure(size=(1000, 1000))

    cols = collect(names(df_all))
    for (i, col) in enumerate(cols)
        ax = Axis(fig[(i-1) % 6 + 1, (i-1) ÷ 6 + 1], 
            xlabel = plot_labels[Symbol(col)],
            ylabel = "counts"
            )
    
        hist!(df_all[:, col])
    end
    
    fig
end

In [ ]:
plot_hists(df_all)

In [ ]:
function plot_scatters(df_all, y_col, ylabel)
    fig = Figure(size=(1600, 1000))
    plot_kwargs = (;
        color=:black, alpha=0.1,markersize=5
        )

    cols = collect(names(df_all))

    for (i, col) in enumerate(cols)
        ax = Axis(fig[(i-1) ÷ 6 + 1, (i-1) % 6 + 1], 
            xlabel = plot_labels[Symbol(col)],
            ylabel = ylabel
            )
    
        scatter!(df_all[:, col], df_all[:, y_col]; plot_kwargs... )
        LilGuys.hide_grid!(ax)
    end
    
    fig
end

In [ ]:
function plot_all(df_all)

        println("perilmc: ", LilGuys.quantile(df_all.peri_lmc, [0.001, 0.16, 0.5, 0.84, 0.999]))
        println("peri: ", LilGuys.quantile(df_all.peri, [0.001, 0.16, 0.5, 0.84, 0.999]))
    
    plot_scatters(df_all, :peri, "peri MW") |> display
    plot_scatters(df_all, :peri_lmc, "peri lmc") |> display
    plot_scatters(df_all, :t_peri, "t peri MW") |> display
    plot_scatters(df_all, :t_peri_lmc, "t peri lmc") |> display

end

In [ ]:
plot_all(df_all)

### Looser errors

In [ ]:
icrs_err.ra

In [ ]:
icrs_lmc_me = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = 0.229,
    radial_velocity = 262,
    distance = 49.59
    )

err_scale = 3
icrs_lmc_me_err = ICRS(
    ra=err_scale * 0.3,
    dec=err_scale * 0.3,
    distance=err_scale * 0.54,
    pmra=err_scale * 0.2,
    pmdec=err_scale * 0.47,
    radial_velocity=err_scale * 3,
    )
icrs_err = ICRS(
    ra=err_scale * 0.01,
    dec=err_scale * 0.01,
    pmra=err_scale * 0.03,
    pmdec=err_scale * 0.03,
    radial_velocity=err_scale * 1,
    distance=err_scale * 2
    )


In [ ]:
orbits, orbits_scl, orbits_scl_lmc, Φs, df_all = scl_lmc_sample_and_orbit(pot_frozen, icrs, icrs_err, icrs_lmc_me, icrs_lmc_me_err, N=1000)

In [ ]:
plot_orbits(orbits, 100)

In [ ]:
plot_orbits(orbits_scl, 100)

In [ ]:
plot_orbits(orbits_scl_lmc, 100)

In [ ]:
plot_hists(df_all)

In [ ]:
x = df_all.Mlmc
y = [minimum(calc_r(o.position)) for o in orbits]
scatter(log10.(x), y, alpha=0.2, markersize=5)

In [ ]:
plot_all(df_all)

### Investigating close encounters

In [ ]:
icrs_lmc_me = ICRS(ra = 80.8,
    dec = -69.8, 
    pmra = 1.910,
    pmdec = -0.4,
    radial_velocity = 262,
    distance = 49.59
    )

err_scale = 1
icrs_lmc_me_err = ICRS(
    ra=err_scale * 0.3,
    dec=err_scale * 0.3,
    distance=err_scale * 0.54,
    pmra=err_scale * 0.2,
    pmdec=err_scale * 0.47,
    radial_velocity=err_scale * 3,
    )
icrs_err = ICRS(
    ra=err_scale * 0.01,
    dec=err_scale * 0.01,
    pmra=err_scale * 0.03,
    pmdec=err_scale * 0.03,
    radial_velocity=err_scale * 1,
    distance=err_scale * 2
    )


In [ ]:
orbits, orbits_scl, orbits_scl_lmc, Φs, df_all = scl_lmc_sample_and_orbit(pot_frozen, icrs, icrs_err, icrs_lmc_me, icrs_lmc_me_err, N=1000)

In [ ]:
plot_orbits(orbits, 100)

In [ ]:
plot_orbits(orbits_scl, 100)

In [ ]:
plot_orbits(orbits_scl_lmc, 100)

In [ ]:
plot_hists(df_all)

In [ ]:
x = df_all.Mlmc
y = [minimum(calc_r(o.position)) for o in orbits]
scatter(log10.(x), y, alpha=0.2, markersize=5)

In [ ]:
plot_all(df_all[df_all.peri_lmc .< 20, :])